In [1]:
import os
import sys
import random
import numpy as np
from PIL import Image, ImageFile
import face_recognition
import glob
from tqdm import tqdm

ROOT_DIR = os.path.abspath(os.curdir)

# Set seed to sample same set of images each time
random.seed(61)

# We need nose bridge and chin to fit mask on a face
KEY_FACIAL_FEATURES = {'nose_bridge', 'chin'}
MODEL = 'cnn' # cnn or hog cnn is slower than hog but more accurate in terms of face detection 
ROOT_DIR = os.path.dirname(os.path.abspath(os.curdir))

In [2]:
def create_masked_face(image_path, mask_path):
    # Convert image into format that face_recognition library understands 
    face_image_np = face_recognition.load_image_file(image_path)
    
    # Recognize face boundaries from an image 
    face_locations = face_recognition.face_locations(face_image_np, model=MODEL)
    
    # Find facial landmarks from the recognized face to fit mask
    face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations)
    has_key_face_landmarks = check_face_landmarks(face_landmarks)
    
    if has_key_face_landmarks:
        face_img = Image.fromarray(face_image_np)
        mask_img = Image.open(mask_path)
        face_mask_img = mask_face(face_img, mask_img, face_landmarks[0])
        cropped_face_mask_img = crop_image(face_mask_img, face_locations[0])
        return cropped_face_mask_img
    else:
        return None

def check_face_landmarks(face_landmarks):
    # Check whether there is a face_landmark
    if len(face_landmarks) > 0:
        # Check whether face_landmarks include all key facial features to fit mask
        if face_landmarks[0].keys() >= KEY_FACIAL_FEATURES:
            return True
        else:
            return False
    else:
        return False

def mask_face(face_img, mask_img, face_landmark):
    nose_bridge = face_landmark['nose_bridge']
    nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
    nose_v = np.array(nose_point)

    chin = face_landmark['chin']
    chin_len = len(chin)
    chin_bottom_point = chin[chin_len // 2]
    chin_bottom_v = np.array(chin_bottom_point)
    chin_left_point = chin[chin_len // 8]
    chin_right_point = chin[chin_len * 7 // 8]

    # split mask and resize
    width = mask_img.width
    height = mask_img.height
    width_ratio = 1.2
    new_height = int(np.linalg.norm(nose_v - chin_bottom_v))

    # left
    mask_left_img = mask_img.crop((0, 0, width // 2, height))
    mask_left_width = get_distance_from_point_to_line(chin_left_point, nose_point, chin_bottom_point)
    mask_left_width = int(mask_left_width * width_ratio)
    mask_left_img = mask_left_img.resize((mask_left_width, new_height))

    # right
    mask_right_img = mask_img.crop((width // 2, 0, width, height))
    mask_right_width = get_distance_from_point_to_line(chin_right_point, nose_point, chin_bottom_point)
    mask_right_width = int(mask_right_width * width_ratio)
    mask_right_img = mask_right_img.resize((mask_right_width, new_height))

    # merge mask
    size = (mask_left_img.width + mask_right_img.width, new_height)
    mask_img = Image.new('RGBA', size)
    mask_img.paste(mask_left_img, (0, 0), mask_left_img)
    mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

    # rotate mask
    angle = np.arctan2(chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
    rotated_mask_img = mask_img.rotate(angle, expand=True)

    # calculate mask location
    center_x = (nose_point[0] + chin_bottom_point[0]) // 2
    center_y = (nose_point[1] + chin_bottom_point[1]) // 2

    offset = mask_img.width // 2 - mask_left_img.width
    radian = angle * np.pi / 180
    box_x = center_x + int(offset * np.cos(radian)) - rotated_mask_img.width // 2
    box_y = center_y + int(offset * np.sin(radian)) - rotated_mask_img.height // 2

    # add mask
    face_img.paste(mask_img, (box_x, box_y), mask_img)
    return face_img

def get_distance_from_point_to_line(point, line_point1, line_point2):
    distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                      (line_point1[0] - line_point2[0]) * point[1] +
                      (line_point2[0] - line_point1[0]) * line_point1[1] +
                      (line_point1[1] - line_point2[1]) * line_point1[0]) / \
               np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                       (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
    return int(distance)

def save(save_dir, fname, face_img):
    dest_path = os.path.join(save_dir, fname)
    face_img.save(dest_path)
    
def crop_image(img, face_location):
    top, right, bottom, left = face_location
    return img.crop((left, top, right, bottom))

In [3]:
available_masks = glob.glob(os.path.join(ROOT_DIR, 'data', 'mask-templates', "*.png"))

available_sample_face_imgs = glob.glob(os.path.join(ROOT_DIR, 'data', 'sampled_face_images', "*.jpg"))
random.shuffle(available_sample_face_imgs)
target_n_masked_face_imgs = len(available_sample_face_imgs) / 2
n_masked_images = 0

# Create a directory to save masked / not masked images
masked_img_dir = os.path.join(ROOT_DIR, 'data', 'train', 'masked')
not_masked_img_dir = os.path.join(ROOT_DIR, 'data', 'train', 'not_masked')
os.makedirs(os.path.join(ROOT_DIR, 'data', 'train'), exist_ok=True)
os.makedirs(masked_img_dir, exist_ok=True)
os.makedirs(not_masked_img_dir, exist_ok=True)

pbar = tqdm(total=target_n_masked_face_imgs)
while n_masked_images < target_n_masked_face_imgs:
    image_path = available_sample_face_imgs.pop()
    fname  = os.path.basename(image_path)
    
    random_mask_path = random.choice(available_masks)
    try:
        masked_face = create_masked_face(image_path, random_mask_path)
    except:
        print('{} file is passed'.format(fname))
    if masked_face is not None:
        save(masked_img_dir, fname, masked_face)
        n_masked_images += 1
        pbar.update(1)
pbar.close()

 24%|██▍       | 601/2500.0 [06:44<16:05,  1.97it/s]  

n009028_0220_02.jpg file is passed


100%|██████████| 2500/2500.0 [30:21<00:00,  1.37it/s] 


In [4]:
for remaining_img_path in tqdm(available_sample_face_imgs):
    # Convert image into format that face_recognition library understands 
    face_image_np = face_recognition.load_image_file(remaining_img_path)
    
    # Recognize face boundaries from an image 
    face_locations = face_recognition.face_locations(face_image_np, model=MODEL)

    img = Image.open(remaining_img_path)
    if len(face_locations) > 0:
        cropped_img = crop_image(img, face_locations[0])
        fname = os.path.basename(remaining_img_path)
        save(not_masked_img_dir, fname, cropped_img)

100%|██████████| 2467/2467 [26:51<00:00,  1.53it/s] 
